In [1]:
import numpy as np
from itertools import product
import rasterio
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## 1. Sample 2D array

In [7]:
A = np.array(range(0,25)).reshape(5,5).astype(float)
A

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]])

### Padding

In [8]:
np.pad(A, pad_width=1, mode="constant", constant_values=np.nan)

array([[nan, nan, nan, nan, nan, nan, nan],
       [nan,  0.,  1.,  2.,  3.,  4., nan],
       [nan,  5.,  6.,  7.,  8.,  9., nan],
       [nan, 10., 11., 12., 13., 14., nan],
       [nan, 15., 16., 17., 18., 19., nan],
       [nan, 20., 21., 22., 23., 24., nan],
       [nan, nan, nan, nan, nan, nan, nan]])

## Mapping values with indices

Indices of adjacent values in anti-clockwise direction:

1. top-left = 0,0
2. left = 1,0
3. bottom-left = 2,0
4. bottom = 2,1
5. bottom-right = 2,2
6. right = 1,2
7. top-right = 0,2
8. top = 0,1

In [14]:
index = (2,1)
B = np.pad(A,pad_width=1, constant_values=np.nan)[index[0]:index[0]+A.shape[0],index[1]:index[1]+A.shape[1]]
print(B.shape)
B

(5, 5)


array([[ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.],
       [nan, nan, nan, nan, nan]])

In [15]:
A-B

array([[-5., -5., -5., -5., -5.],
       [-5., -5., -5., -5., -5.],
       [-5., -5., -5., -5., -5.],
       [-5., -5., -5., -5., -5.],
       [nan, nan, nan, nan, nan]])

## Loading all the indices 

from top-left to top adjancent values in anti-clockwise direction

In [4]:
p = product([0,1,2],repeat=2)
indices = []
for x in p:
    if x != (1,1):
        indices.append(x)
        
print(indices)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)]


## Calculating the slope

In [5]:
def adjacent_values(A, index=(1,1)):
    B = np.pad(A,pad_width=1)[index[0]:index[0]+A.shape[0],index[1]:index[1]+A.shape[1]]
    return B

In [6]:
slope = np.zeros(A.shape)
for i in indices:
    B = adjacent_values(A, index=i)
    slope += A - B

In [7]:
slope

array([[-12., -12.,  -9.,  -6.,  12.],
       [ 12.,   0.,   0.,   0.,  30.],
       [ 27.,   0.,   0.,   0.,  45.],
       [ 42.,   0.,   0.,   0.,  60.],
       [108.,  78.,  81.,  84., 132.]])

## Calculating slope for very big matrix

In [5]:
def adjacent_values(A, index=(1,1)):
    B = np.pad(A,pad_width=1)[index[0]:index[0]+A.shape[0],index[1]:index[1]+A.shape[1]]
    return B

def vectorized_slope_calc(A):   
    
    '''
    Calculates the slope of a value in a matrix with respect to its neighboring values.
    Currently the kernel size chosen is around 3X3 matrix. 
    
    INPUT: takes a matrix of which slope is to be calculated
    Returns: the slope matrix of the same size as the input matrix
    '''
    if A.shape[0] >= 3 and A.shape[0]==A.shape[1]:
        ## List down all the indices of the adjacent values:
        p = product(list(range(0,3)),repeat=2) ## Kernel size taken as 3
        indices = []
        for x in p:
            if x != (1,1):
                indices.append(x)

        ## Calculate the slope:
        slope = np.zeros(A.shape)
        for i in indices:
            B = adjacent_values(A, index=i) ## Find the adjacent matrix
            slope += np.abs(A - B)  ## Only find the magnitude of the difference

        return slope
    
    else:
        print("Array size too small or not a square matrix. It should be greater than or equal to (3,3)")
        return None

In [9]:
%%time
X = np.int32(np.random.randn(5,5)*100)
print(X)
vectorized_slope_calc(X)

[[ 164 -147 -151  -38   38]
 [ 114 -186  -62   24   28]
 [  42   54  -55  244  134]
 [  62   42  -42 -132   -7]
 [ -29  151  -38 -221  181]]
Wall time: 9.56 ms


array([[1531., 1141.,  869.,  455.,  290.],
       [1085., 1447.,  837.,  750.,  486.],
       [ 458.,  653.,  812., 2064., 1135.],
       [ 414.,  473.,  945., 1430.,  940.],
       [ 487., 1213.,  664., 1730., 1808.]])

In [10]:
%%time
Y = np.random.randn(5000,5000)
print((Y.size * Y.itemsize)/1024**3, "GB")
# slope_vectorized(X)

0.1862645149230957 GB
Wall time: 4.31 s


**Some Statistics**
1. Matrix size 10000 by 10000, memory size=0.74 GB | Time taken : 9 seconds
2. Matrix size 20000 by 20000, memory size=3.0 GB | Time taken : 3 mins

In [11]:
%%time
S = vectorized_slope_calc(Y)
print(S.shape)
S

(5000, 5000)
Wall time: 9.91 s


array([[ 5.30081995,  4.32876667,  4.46221476, ...,  5.01434163,
         7.92927834, 15.23068095],
       [ 4.88473814, 11.6689104 ,  7.98231028, ...,  6.95144713,
         8.1354536 , 11.33454662],
       [ 5.80198529,  7.34073471, 15.48076382, ...,  6.98245503,
         6.92152771,  9.42622534],
       ...,
       [ 7.05487753, 14.67206115, 13.47647036, ..., 12.99319906,
        12.87836391,  7.01797595],
       [ 6.47649235, 10.70006632,  8.66281141, ..., 12.28129229,
        15.74930557,  6.05783338],
       [ 3.89090199,  3.86018761,  7.46418244, ...,  7.68141404,
        11.15461709,  6.95458439]])

## Scaling the outputs
Scaling the outputs in between 0,1

In [12]:
S.min(), S.max()

(0.7114276804899553, 48.98780829679346)

In [13]:
scaler = MinMaxScaler()
S_sc = scaler.fit_transform(S)
S_sc.min(), S_sc.max()

(0.0, 1.0000000000000002)

In [15]:
xi,yi = np.where(S_sc>=0.4)
data = np.column_stack((xi,yi,S[xi,yi]))
data = pd.DataFrame(data, columns=["x","y","z"])
data[["x","y"]] = data[["x","y"]].astype(np.uint16)
data

,x,y,z
0,0,12,12.691395
1,0,35,14.998490
2,0,51,18.462999
3,0,93,14.793876
4,0,94,16.113156
...,...,...,...
2633216,4999,4978,15.072352
2633217,4999,4979,18.179902
2633218,4999,4982,15.695915
2633219,4999,4988,15.016019


In [16]:
## find the grid points
grid_size = 20
ai = np.arange(0,S.shape[0],grid_size)
bi = np.arange(0,S.shape[1],grid_size)
Ai, Bi = np.meshgrid(ai,bi)
grid = S[Ai,Bi]
grid = np.column_stack((Ai.ravel(), Bi.ravel(), grid.ravel()))
grid = pd.DataFrame(grid, columns=["x","y","z"])
grid[["x","y"]] = grid[["x","y"]].astype(np.uint16)

output = pd.concat([grid, data]).drop_duplicates(subset=["x","y"])
output

,x,y,z
0,0,0,5.300820
1,20,0,5.354046
2,40,0,8.143778
3,60,0,8.789784
4,80,0,13.708069
...,...,...,...
2633216,4999,4978,15.072352
2633217,4999,4979,18.179902
2633218,4999,4982,15.695915
2633219,4999,4988,15.016019


## Improvements

For arrays which are out of memory when loaded, `dask.arrays` can be used for overcoming this situation. Also with this we can further improve the calculating by implementing parrallel computing using `dask`.